In [1]:
import pandas as pd
import numpy as np


In [2]:
# Generate random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# When this cell is executed, the output is a zip object in memory
# The zip object packs each pair of lats and longs having the same index in their respective array
# into a tuple

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [5]:
import sys  
!{sys.executable} -m pip install --user citipy

  Using cached citipy-0.0.5.tar.gz (557 kB)
  Using cached kdtree-0.16-py2.py3-none-any.whl (7.7 kB)
  Created wheel for citipy: filename=citipy-0.0.5-py3-none-any.whl size=559700 sha256=5923c3046e604f7458f50bd2af997022ceafc7a193813906ac39029341b01ba9
  Stored in directory: /Users/benjaminaltshuler/Library/Caches/pip/wheels/72/ae/63/5d03cf6dfe5c0e9328fb73f8b30da1948a06b9cceaf2c68d2d
Successfully built citipy


In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

723

In [16]:
# Import the requests library
import requests

# Import the API key
from config import weather_api_key

In [10]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=64056757d6f3a8371d9fa8d2c9180513


In [11]:
# Import the datetime module from the datetime library
from datetime import datetime

In [12]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning of data retrieval          ")
print("-------------------------------------")

# Create the counters and initialize to 1 because the first iteration of logging for each recorded response and
# the set needs to start at 1
record_count = 1
set_count = 1


# Every time a city is referenced in the code, the index on the cities list needs to be used
# This will cause programming errors when building city_url because it adds to the index, not the city name to
# city_url
# To fix this issue, another for loop needs to be created in order to add the city from the cities list
# Instead of using two for loops, use the enumerate() method as an alternative way to iterate through the list 
# of cities and retrieve both the index and city from the list

# Loop through the cities in the list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1
    
# Add a try-except block to prevent API requests from stopping prematurelyy if the city_weather requests is 
# not a valuid request
# If the request isn't valid, the code will not find the first item requested, which is the dictionary "coord"
# The code will skip the city and continue to run

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve the data
        city_weather = requests.get(city_url).json()

        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Description": city_description})
        
# If an error is given, skip the city
    except:
        print("City not found.  Skipping...")
        pass
    
# Indicate the date loading is complete
print("------------------------------")
print("Date retrieval complete       ")
print("------------------------------")

Beginning of data retrieval          
-------------------------------------
Processing Record 1 of Set 1 | barentsburg
City not found.  Skipping...
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | black river
Processing Record 4 of Set 1 | clyde river
Processing Record 5 of Set 1 | alekseyevsk
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | avarua
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | ust-kamchatsk
City not found.  Skipping...
Processing Record 11 of Set 1 | los llanos de aridane
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | amderma
City not found.  Skipping...
Processing Record 14 of Set 1 | xixiang
Processing Record 15 of Set 1 | vaini
Processing Record 16 of Set 1 | sao filipe
Processing Record 17 of Set 1 | cape town
Processing Record 18 of Set 1 | qaanaaq
Processing Record 19 of Set 1 | busselton
Processing Record 20 of Set 1 | thompson
Processi

In [13]:
# Print the city count to confirm sufficient count.
len(city_data)

659

In [14]:

# Convert the array of dictionaries to a DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,Mataura,-46.1927,168.8643,70.75,52,100,4.85,NZ,2021-10-25 00:48:32,overcast clouds
1,Black River,18.0264,-77.8487,82.40,80,37,4.85,JM,2021-10-25 00:48:33,scattered clouds
2,Clyde River,70.4692,-68.5914,12.87,77,97,4.65,CA,2021-10-25 00:47:15,overcast clouds
3,Alekseyevsk,57.8406,108.3492,39.22,91,100,12.53,RU,2021-10-25 00:48:33,overcast clouds
4,Bluff,-46.6000,168.3333,57.72,81,99,11.92,NZ,2021-10-25 00:48:33,overcast clouds
5,Avarua,-21.2078,-159.7750,77.05,53,20,12.66,CK,2021-10-25 00:48:34,few clouds
6,Albany,42.6001,-73.9662,51.80,84,96,2.08,US,2021-10-25 00:48:34,overcast clouds
7,Ushuaia,-54.8000,-68.3000,33.46,80,75,28.77,AR,2021-10-25 00:44:36,light snow
8,Los Llanos De Aridane,28.6585,-17.9182,67.75,88,90,8.05,ES,2021-10-25 00:48:35,overcast clouds
9,Rikitea,-23.1203,-134.9692,74.79,76,25,17.40,PF,2021-10-25 00:46:29,scattered clouds


In [15]:

# Create the CSV output file
output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# Export city_data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")